In [1]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from tools.standard_name_utils import optimize_parameters
from tools.common_utils import highlight_diff, read_jsonl

model_path = "../../train-result/2024-06-19/checkpoint-5000"
top_p=0.7 
temperature=0.95

/home/reymond/anaconda3/envs/afac2024/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()
model = model.eval()

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
Loading checkpoint shards: 100%|██████████| 6/6 [11:36<00:00, 116.11s/it]


In [3]:
# response, history = model.chat(tokenizer, "你好", history=[], top_p=top_p, temperature=temperature)
# print(response)

In [8]:
json_file_path = 'data/train_2.json'
data_list = read_jsonl(json_file_path)

df = pd.DataFrame(data_list)
df.head()

,conversations
0,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
1,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
2,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
3,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."
4,"[{'role': 'user', 'content': '你现在是一个金融领域专家，你需要..."


In [10]:
df.shape

(4460, 1)

In [12]:
total_eval_count = 0.0
correct_count = 0.0

result_arr = []

for index, row in df.iterrows():    
    single_user_row = row["conversations"][0]
    single_assis_row = row["conversations"][1]
    
    input = single_user_row["content"]
    output = single_assis_row["content"]
    
    response, history = model.chat(tokenizer=tokenizer, query=input, history=[], top_p=top_p, temperature=temperature)
    
    # optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)
    
    total_eval_count += 1
    
    result_data = {}
    result_data["query"] = input
    result_data["label"] = output
    result_data["predict"] = response
    result_arr.append(result_data)
    
    # if response == output:
    #     correct_count += 1
    # else:
    #     print("发现一条错误数据，index：" + f"{index}")
    #     err_data = {}
    #     err_data["query"] = input
    #     err_data["label"] = output
    #     err_data["predict"] = response
    #     result_arr.append(err_data)
        # print("-----data index-----")
        # print(index)
        # print("-----query input-----")
        # print(input)
        # print("-----output diff-----")
        # print(highlight_diff(output, response))
        # print(response)
        # print()
    
    if total_eval_count % 10 == 0:
        print("现在是第" + f"{total_eval_count}" + "条数据")
    
# print("预测正确的比例：" + f"{correct_count / total_eval_count :.2%}")

TypeError: 'module' object is not callable

In [6]:
result_df = pd.DataFrame(result_arr)
result_df.head(5)

,query,label,predict
0,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":...","{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
1,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":...","{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
2,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":...","{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
3,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""tool_name""...","{""relevant APIs"": [{""api_id"": ""0"", ""tool_name""..."
4,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""tool_name""...","{""relevant APIs"": [{""api_id"": ""0"", ""tool_name""..."


In [7]:
result_df.to_csv("./data/train_result_1.csv")

In [ ]:
# train_err_df_1 = pd.read_csv("./data/train_err_1.csv")
# train_err_df_2 = pd.read_csv("./data/train_err_2.csv")
# train_err_df_3 = pd.read_csv("./data/train_err_3.csv")
# train_err_df_4 = pd.read_csv("./data/train_err_4.csv")
# train_err_df_5 = pd.read_csv("./data/train_err_5.csv")
# train_err_df_6 = pd.read_csv("./data/train_err_6.csv")
# train_err_df_7 = pd.read_csv("./data/train_err_7.csv")
# train_err_df_8 = pd.read_csv("./data/train_err_8.csv")
# train_err_df_9 = pd.read_csv("./data/train_err_9.csv")
# train_err_df_10 = pd.read_csv("./data/train_err_10.csv")

# train_err_df = pd.concat([
#     train_err_df_1, 
#     train_err_df_2, 
#     train_err_df_3, 
#     train_err_df_4, 
#     train_err_df_5, 
#     train_err_df_6, 
#     train_err_df_7, 
#     train_err_df_8, 
#     train_err_df_9, 
#     train_err_df_10])

In [ ]:
# train_err_df.head()

In [ ]:
# train_err_df.to_csv("./data/train_err.csv")